In [1]:
import pandas as pd
import numpy as np
import openai
import ast
import sys
import json
pd.set_option('display.max_columns', 200)

from dotenv import load_dotenv
import os
load_dotenv("../config.env")

True

## **Carga del dataframe:**

In [2]:
df = pd.read_csv('df.csv')

## **Transformaciones:**

### Belong to collection

In [3]:
#transformar a diccionarios:
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else {'data': None})

La mayor cantidad de belong to collection esta vacia, probaré dejandola a ver como quedan los vectores, solo voy a usar 'name':'

In [4]:
df['belongs_to_collection'][0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

Defino una nueva columna útil para el modelo

In [5]:
df['collection'] = df['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else '')

### Genres

In [6]:
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else {'data': None})

In [7]:
df['genres'][0]

{'genres': [{'id': 16, 'name': 'Animation'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 10751, 'name': 'Family'}]}

In [8]:
df['joined_genres'] = df['genres'].apply(lambda x: ' '.join([genre['name'] for genre in x['genres']]) if isinstance(x, dict) and 'genres' in x else '')

#Por ahora no voy a trabajar con production company ni country

### Cast


Sólo tomaré los nombres de los actores

In [9]:
df['cast_names'] = df['cast'].apply(lambda x: ' '.join([actor['name'] for actor in ast.literal_eval(x)]) if isinstance(x, str) else '')

In [10]:
df['cast'][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

### Crew


Solo tomaré el director

In [11]:
import math

def convert_to_json(texto):
    if pd.isna(texto) or (isinstance(texto, float) and math.isnan(texto)):
        return ''  # Reemplazar NaN por cadena vacía
    
    if isinstance(texto, str):
        try:
            estructura_datos = ast.literal_eval(texto)
            if isinstance(estructura_datos, (dict, list)):
                texto_json = json.dumps(estructura_datos)
                objeto_json = json.loads(texto_json)
                return objeto_json
            else:
                raise ValueError("El contenido no es una estructura de datos válida")
        except (SyntaxError, ValueError) as e:
            raise ValueError("El contenido no se puede evaluar correctamente") from e
    else:
        raise ValueError("El contenido no es una cadena de texto")

In [12]:
df['crew'] = df['crew'].apply(convert_to_json)

In [13]:
df['crew'][0]

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f55',
  'department': 'Writing',
  'gender': 2,
  'id': 7,
  'job': 'Screenplay',
  'name': 'Andrew Stanton',
  'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f5b',
  'department': 'Writing',
  'gender': 2,
  'id': 12892,
  'job': 'Screenplay',
  'name': 'Joel Cohen',
  'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f61',
  'department': 'Writing',
  'gender': 0,
  'id': 12893,
  'job': 'Screenplay',
  'name': 'Alec Sokolow',
  'profile_path': '/v79vlRYi94BZUQnkkyzn

In [14]:
# Crear la nueva columna "director"
df['director'] = ''

# Recorrer cada fila del DataFrame
for index, row in df.iterrows():
    crew = row['crew']
    director_name = ''
    if crew != '':
        for entry in crew:
            if entry['job'] == 'Director':
                director_name = entry['name']
                break
    df.at[index, 'director'] = director_name

In [15]:
df.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew,collection,joined_genres,cast_names,director
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"{'genres': [{'id': 16, 'name': 'Animation'}, {...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,{'production_companies': [{'name': 'Pixar Anim...,"{'production_countries': [{'iso_3166_1': 'US',...",1995-10-30,373554050.0,81.0,"{'spoken_languages': [{'iso_639_1': 'en', 'nam...",Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",Toy Story Collection,Animation Comedy Family,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter


## Redimensionar a las features que serán usadas:

In [16]:

data = df[['id','title','overview','release_year','collection','joined_genres','cast_names','director']]

In [17]:
data[data==''].count()

id                   0
title                0
overview             0
release_year         0
collection       40861
joined_genres     2384
cast_names        2349
director           836
dtype: int64

#### Por limitaciones tecnicas será necesario hacer un muestro de los datos

In [18]:
# Establecer la semilla (seed)
seed = 33
np.random.seed(seed)

# Realizar el muestreo aleatorio
sample_df = data.sample(frac=0.2) 

sample_df = sample_df.reset_index(drop=True)

In [113]:
sample_df.to_csv('paraprobar.csv')

In [19]:
sample_df.head()

,id,title,overview,release_year,collection,joined_genres,cast_names,director
0,55531,The Radio Burglary,Eager young radio reporter Teräsvuori stages a...,1951,,Comedy Crime Thriller,Hannes Häyrinen Ritva Arvelo Kullervo Kalske K...,Matti Kassila
1,104044,Roller Blade Warriors: Taken by Force,"In the future, a warrior nun on roller skates ...",1989,,Fantasy Science Fiction,Kathleen Kinmont Rory Calhoun Cleve Hall Jack ...,Donald G. Jackson
2,30666,Stepfather II: Make Room For Daddy,The Stepfather escapes an insane asylum and wi...,1989,The Stepfather Collection,Horror Thriller,Terry O'Quinn Meg Foster Caroline Williams Jon...,Jeff Burr
3,50936,Moon Warriors,"A kind-hearted fisherman, content with simple ...",1992,,Action Adventure Drama Fantasy Foreign Romance...,Andy Lau Anita Mui Kenny Bee Maggie Cheung Kel...,Sammo Hung
4,108632,The Show,Cock Robin (John Gilbert) is the swaggering ba...,1927,,Crime Drama,John Gilbert Renée Adorée Lionel Barrymore Edw...,Tod Browning


## **Feature engineering** (esto no va)

In [ ]:
#partir el dataset en varios para poder pasarlo a openai

In [89]:
sample_df.shape

(5442, 26)

In [87]:
token = os.getenv("TOKEN")
openai.api_key  = token

In [92]:
#Tomado del curso de # Prompt Engineer de deeplearning.ia-Inferring by AndrewNg / Isa Fulford) 

'''def get_completion(x, model="gpt-3.5-turbo"):
    prompt = f""" 
    Analyze the following movie overview: '''{x}'''. Please generate a list of up to 6 keywords that accurately describe the content and main themes of the movie. Exclude common character names and focus on more general and relevant aspects, the return must be like this: 'keyword1 keyword2...keyword6'. If no keywords are identified, please return ''.
    """ 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
'''import time

# Dividir el DataFrame original en 10 partes
num_parts = 40
df_parts = np.array_split(sample_df, num_parts)

# Realizar operaciones en cada parte del DataFrame
for i, df_part in enumerate(df_parts):

    df_part["Nueva_Columna"] = df_part['overview'].apply(lambda x: get_completion(x))
    print('procesado')
    # Agregar un tiempo de espera de 30 segundos
    time.sleep(120)
# Unir los DataFrames
merged_df = pd.concat(df_parts)

# Imprimir el número de registros en el DataFrame original y el DataFrame unido
print(f"Original DataFrame: {len(sample_df)} registros")
print(f"Merged DataFrame: {len(merged_df)} registros")'''

#continuamos acá:

In [20]:
sample_df.head()

,id,title,overview,release_year,collection,joined_genres,cast_names,director
0,55531,The Radio Burglary,Eager young radio reporter Teräsvuori stages a...,1951,,Comedy Crime Thriller,Hannes Häyrinen Ritva Arvelo Kullervo Kalske K...,Matti Kassila
1,104044,Roller Blade Warriors: Taken by Force,"In the future, a warrior nun on roller skates ...",1989,,Fantasy Science Fiction,Kathleen Kinmont Rory Calhoun Cleve Hall Jack ...,Donald G. Jackson
2,30666,Stepfather II: Make Room For Daddy,The Stepfather escapes an insane asylum and wi...,1989,The Stepfather Collection,Horror Thriller,Terry O'Quinn Meg Foster Caroline Williams Jon...,Jeff Burr
3,50936,Moon Warriors,"A kind-hearted fisherman, content with simple ...",1992,,Action Adventure Drama Fantasy Foreign Romance...,Andy Lau Anita Mui Kenny Bee Maggie Cheung Kel...,Sammo Hung
4,108632,The Show,Cock Robin (John Gilbert) is the swaggering ba...,1927,,Crime Drama,John Gilbert Renée Adorée Lionel Barrymore Edw...,Tod Browning


In [21]:
combined_features = pd.DataFrame()

In [22]:
combined_features['id'] = sample_df['id']
combined_features['text'] = sample_df['title'] + ' ' + sample_df['overview'] + ' ' + sample_df['release_year'].astype(str) + ' ' + sample_df['collection'] + ' ' + sample_df['joined_genres'] + ' ' + sample_df['cast_names'] + ' ' + sample_df['director']

In [23]:
combined_features

,id,text
0,55531,The Radio Burglary Eager young radio reporter ...
1,104044,Roller Blade Warriors: Taken by Force In the f...
2,30666,Stepfather II: Make Room For Daddy The Stepfat...
3,50936,"Moon Warriors A kind-hearted fisherman, conten..."
4,108632,The Show Cock Robin (John Gilbert) is the swag...
...,...,...
9064,411632,End of a Gun A mall security guard -- and form...
9065,42872,Alien 2: On Earth A group of cave explorers ar...
9066,361131,Hate Story 3 The story is about Aditya and Siy...
9067,56763,The South Shaolin Master The narrative centres...


## Limpieza

In [24]:
import re
from unidecode import unidecode

def limpiar_texto(texto):
    texto_sin_saltos = texto.replace("\n", " ")
    texto_minusculas = texto_sin_saltos.lower()
    texto_sin_tildes = unidecode(texto_minusculas)
    patron = r'@[\w]+|#\w+|[!,".]|(\b[^\w\s]\b)|\bhttps?\S+\b'
    texto_limpio = re.sub(patron, "", texto_sin_tildes)
    return texto_limpio

In [25]:
combined_features['cleaned_text'] = combined_features['text'].astype(str).apply(limpiar_texto)

In [26]:
combined_features

,id,text,cleaned_text
0,55531,The Radio Burglary Eager young radio reporter ...,the radio burglary eager young radio reporter ...
1,104044,Roller Blade Warriors: Taken by Force In the f...,roller blade warriors: taken by force in the f...
2,30666,Stepfather II: Make Room For Daddy The Stepfat...,stepfather ii: make room for daddy the stepfat...
3,50936,"Moon Warriors A kind-hearted fisherman, conten...",moon warriors a kindhearted fisherman content ...
4,108632,The Show Cock Robin (John Gilbert) is the swag...,the show cock robin (john gilbert) is the swag...
...,...,...,...
9064,411632,End of a Gun A mall security guard -- and form...,end of a gun a mall security guard -- and form...
9065,42872,Alien 2: On Earth A group of cave explorers ar...,alien 2: on earth a group of cave explorers ar...
9066,361131,Hate Story 3 The story is about Aditya and Siy...,hate story 3 the story is about aditya and siy...
9067,56763,The South Shaolin Master The narrative centres...,the south shaolin master the narrative centres...


In [27]:
### Tokenizar y quitar stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abrahan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
from nltk.tokenize import RegexpTokenizer
regexp = RegexpTokenizer('\w+')
combined_features['token'] = combined_features['cleaned_text'].apply(regexp.tokenize)
combined_features.head()

,id,text,cleaned_text,token
0,55531,The Radio Burglary Eager young radio reporter ...,the radio burglary eager young radio reporter ...,"[the, radio, burglary, eager, young, radio, re..."
1,104044,Roller Blade Warriors: Taken by Force In the f...,roller blade warriors: taken by force in the f...,"[roller, blade, warriors, taken, by, force, in..."
2,30666,Stepfather II: Make Room For Daddy The Stepfat...,stepfather ii: make room for daddy the stepfat...,"[stepfather, ii, make, room, for, daddy, the, ..."
3,50936,"Moon Warriors A kind-hearted fisherman, conten...",moon warriors a kindhearted fisherman content ...,"[moon, warriors, a, kindhearted, fisherman, co..."
4,108632,The Show Cock Robin (John Gilbert) is the swag...,the show cock robin (john gilbert) is the swag...,"[the, show, cock, robin, john, gilbert, is, th..."


In [29]:
### Quitar stopwords

combined_features['token_no_stopwords'] = combined_features['token'].apply(lambda x: [item for item in x if item not in stopwords])

In [30]:
### Lematización y stemming

from nltk.stem import PorterStemmer

stemmer = PorterStemmer()


In [31]:
combined_features['stemming'] = combined_features['token_no_stopwords'].apply(lambda x: [stemmer.stem(item) for item in x ])

In [32]:
combined_features['stemming_unique'] = combined_features['stemming'].apply(lambda x: list(set(x)))

In [33]:
combined_features['stemming_unique']

0       [eager, later, kullervo, start, savolainen, ma...
1       [fantasi, g, damon, cleve, futur, jackson, ban...
2       [howev, anyon, tri, foster, famili, byrdnether...
3       [tam, fantasi, lau, andi, anita, mui, yiuhung,...
4       [swagger, gertrud, dorothi, andi, sebastian, p...
                              ...                        
9064    [jade, troy, former, guard, crime, florin, pie...
9065    [creatur, barres, alien, judi, mayn, confront,...
9066    [hatr, suddenli, take, control, 2015, relation...
9067    [narr, variou, skill, battl, perform, xu, yanl...
9068    [mia, true, allow, 2015, ravello, everyth, goo...
Name: stemming_unique, Length: 9069, dtype: object

In [74]:
#combined_features['string_ready'] = combined_features['stemming_unique'].apply(lambda words: ' '.join(set(words)))
combined_features['string_ready'] = combined_features['stemming'].apply(lambda words: ' '.join(set(words)))
df_ready = combined_features['string_ready']

In [75]:
df_ready

0       eager later kullervo start savolainen matti ei...
1       fantasi g damon cleve futur jackson band blade...
2       howev anyon tri foster famili byrdnetheri futu...
3       tam fantasi lau andi anita mui yiuhung biuchue...
4       swagger gertrud dorothi andi sebastian palac p...
                              ...                        
9064    jade troy former guard crime florin piersic gr...
9065    creatur barres alien judi mayn confront explor...
9066    hatr suddenli take control 2015 relationship f...
9067    narr variou skill battl perform xu yanlong cen...
9068    mia true allow 2015 ravello everyth good bova ...
Name: string_ready, Length: 9069, dtype: object

In [76]:
type(df_ready)

pandas.core.series.Series

In [37]:
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [77]:
vectorizer = TfidfVectorizer()

In [78]:
feature_vectors = vectorizer.fit_transform(df_ready)

In [79]:
print(feature_vectors)

  (0, 12324)	0.04520356498281607
  (0, 12219)	0.12593494202134622
  (0, 33725)	0.18877117199627821
  (0, 30740)	0.18877117199627821
  (0, 12149)	0.14709854945766473
  (0, 7626)	0.10265230349979604
  (0, 50302)	0.1092053496515397
  (0, 3661)	0.18877117199627821
  (0, 48217)	0.11077794399706142
  (0, 18621)	0.08842347366722174
  (0, 9103)	0.15862901682220962
  (0, 33476)	0.18064553149901283
  (0, 28579)	0.09201033893623527
  (0, 25576)	0.18877117199627821
  (0, 3607)	0.09658265441548342
  (0, 31136)	0.18064553149901283
  (0, 60276)	0.08731527258841944
  (0, 21206)	0.066744914643455
  (0, 13346)	0.1005817880057649
  (0, 13732)	0.1447381426426718
  (0, 22498)	0.06475653162214177
  (0, 66777)	0.058939925711552
  (0, 49825)	0.11248453162265748
  (0, 23121)	0.07683862283567326
  (0, 59932)	0.18877117199627821
  :	:
  (9068, 25149)	0.09785073578241604
  (9068, 23360)	0.08663509224116558
  (9068, 331)	0.07779004892046812
  (9068, 54145)	0.11215094887439167
  (9068, 36532)	0.1270454025068022
  (

In [80]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [81]:
print(similarity.shape)

(9069, 9069)


In [43]:
import pickle

with open('similarity.pickle','wb') as f:
    pickle.dump(similarity,f)

In [102]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

In [103]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = sample_df['title'].tolist()
print(list_of_all_titles)

['The Radio Burglary', 'Roller Blade Warriors: Taken by Force', 'Stepfather II: Make Room For Daddy', 'Moon Warriors', 'The Show', 'Cop', 'I Am Not Your Negro', 'Radio Unnameable', 'Domésticas', 'Série noire', 'Bugcrush', 'Babycall', 'Don King: Only in America', 'My Way', 'The Last Match', 'The Memory Of A Killer', 'Micro Men', "Inside Hana's Suitcase", 'Bangkok Adrenaline', 'The Cool World', "Michael Jackson's Journey from Motown to Off the Wall", 'Sitting Target', 'Bhaji on the Beach', 'Never Back Down 2: The Beatdown', 'Three', 'Ninja 4: Siege', 'The Booth', 'Operation Dumbo Drop', 'American Madness', 'Till Luck Do Us Part', 'Deceptive Practice: The Mysteries and Mentors of Ricky Jay', 'Here Comes the Navy', 'The Sudden Wealth of the Poor People of Kombach', 'Meet Monica Velour', 'Johnny 100 Pesos', 'Bilitis', "Clifford's Really Big Movie", 'The Hunger Games: Catching Fire', 'The Ecstasy of Wilko Johnson', 'Far from Home', "National Lampoon's Vacation", 'Tobacco Road', 'Ip Man', 'Fo

In [104]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Alien 2: On Earth', 'Alien: Covenant']


In [105]:
close_match = find_close_match[0]
print(close_match)

Alien 2: On Earth


In [106]:
# finding the index of the movie with title

index_of_the_movie = sample_df[sample_df['title'] == close_match].index#.values[0]
print(index_of_the_movie)

Index([9065], dtype='int64')


In [107]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, array([0.        , 0.02662419, 0.0046655 , ..., 0.0184488 , 0.00396161,
       0.02910508]))]


In [108]:
len(similarity_score)

1

In [109]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(0, array([0.        , 0.02662419, 0.0046655 , ..., 0.0184488 , 0.00396161,
       0.02910508]))]


In [111]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = sample_df[sample_df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 



TypeError: tuple indices must be integers or slices, not tuple

In [92]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = sample_df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = sample_df[sample_df.title == close_match].index

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = sample_df[sample_df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . The Radio Burglary


In [116]:
sample_df.head(100)

,id,title,overview,release_year,collection,joined_genres,cast_names,director
0,55531,The Radio Burglary,Eager young radio reporter Teräsvuori stages a...,1951,,Comedy Crime Thriller,Hannes Häyrinen Ritva Arvelo Kullervo Kalske K...,Matti Kassila
1,104044,Roller Blade Warriors: Taken by Force,"In the future, a warrior nun on roller skates ...",1989,,Fantasy Science Fiction,Kathleen Kinmont Rory Calhoun Cleve Hall Jack ...,Donald G. Jackson
2,30666,Stepfather II: Make Room For Daddy,The Stepfather escapes an insane asylum and wi...,1989,The Stepfather Collection,Horror Thriller,Terry O'Quinn Meg Foster Caroline Williams Jon...,Jeff Burr
3,50936,Moon Warriors,"A kind-hearted fisherman, content with simple ...",1992,,Action Adventure Drama Fantasy Foreign Romance...,Andy Lau Anita Mui Kenny Bee Maggie Cheung Kel...,Sammo Hung
4,108632,The Show,Cock Robin (John Gilbert) is the swaggering ba...,1927,,Crime Drama,John Gilbert Renée Adorée Lionel Barrymore Edw...,Tod Browning
...,...,...,...,...,...,...,...,...
95,31041,No Greater Love,Jeff and Heather Baker were life long sweethea...,2009,,Documentary Romance,Anthony Tyler Quinn Danielle Bisutti Jay Under...,Brad J. Silverman
96,339327,Industrial Soundtrack for the Urban Decay,Industrial Soundtrack For The Urban Decay trac...,2015,,Documentary Music,,Amélie Ravalec
97,157832,Calvary,"After he is threatened during a confession, a ...",2014,,Drama,Brendan Gleeson Chris O'Dowd Kelly Reilly Aida...,John Michael McDonagh
98,159622,Terms and Conditions May Apply,Have you ever read the Terms and Conditions an...,2013,,Documentary,Leigh Bryan Raymond Kurzweil Joe Lipari Moby M...,Cullen Hoback
